In [1]:
import json
import pandas as pd
import os
import sqlite3
from sqlite3 import Error
from statistics import mean
import numpy as np
import seaborn as sns

directory_with_files = '../data'
db_name = './test.db'
avia_companies = {'KLM': 56377143, 'AirFrance': 106062176, 'British_Airways': 18332190, 'AmericanAir': 22536055,
                  'Lufthansa': 124476322, 
                   'easyJet': 38676903, 'RyanAir': 1542862735, 'SingaporeAir': 253340062,
                  'Qantas': 218730857, 'EtihadAirways': 45621423,
                  'VirginAtlantic': 20626359}

In [2]:
reverse_avia_companies={}
for key in avia_companies.keys():
    reverse_avia_companies[avia_companies[key]]=key

In [3]:
reverse_avia_companies

{56377143: 'KLM',
 106062176: 'AirFrance',
 18332190: 'British_Airways',
 22536055: 'AmericanAir',
 124476322: 'Lufthansa',
 38676903: 'easyJet',
 1542862735: 'RyanAir',
 253340062: 'SingaporeAir',
 218730857: 'Qantas',
 45621423: 'EtihadAirways',
 20626359: 'VirginAtlantic'}

In [4]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

In [5]:
query_for_replies = '''
SELECT tweets.user_id, reply_id, reply_to_status_id, reply_to_user_id, tweet_id, followers_count
FROM tweets, replies, users
WHERE tweet_id = reply_id AND users.user_id = reply_to_user_id
'''
df_all_replies = pd.read_sql_query(query_for_replies, create_connection(db_name))


2.6.0


In [6]:
def get_reply_perc(air_id,avia_companies, df_all_replies, conn):
    df_mentioned = get_df_mentioned(air_id, conn)
    df_answered = get_df_answered(air_id, df_mentioned, df_all_replies)
    return len(df_answered)/len(df_mentioned)

def get_df_mentioned(air_id, conn):
    query = f'''
    SELECT tweets.tweet_id AS uniq, user_mentions.tweet_id, user_mentions.text, tweets.text, tweets.tweet_type, tweets.lang 
    FROM tweets, user_mentions
    WHERE tweets.tweet_id = user_mentions.tweet_id AND user_mentions.text = '{air_id}' AND tweets.tweet_type = 'original' AND tweets.lang != 'und'
    '''
    df= pd.read_sql_query(query, conn).drop_duplicates()
    return df
    
def get_df_answered(air_id, df_mentioned, df_all_replies):
    tweet_ids_mentioned = list(df_mentioned['uniq'].unique())
    df_answer = df_all_replies[(df_all_replies['reply_to_status_id'].isin(tweet_ids_mentioned)) & (df_all_replies['user_id']==air_id)].copy()
    return df_answer

In [7]:
def get_url_perc(air_id,avia_companies, df_all_replies, conn):
    df_mentioned = get_df_mentioned(air_id, conn)
    df_answered = get_df_answered(air_id, df_mentioned, df_all_replies)
    df_url = get_df_url(conn)
    tweets_url = list(df_url['uniq'].unique())
    df_answered_url = df_answered[df_answered['tweet_id'].isin(tweets_url)]
    if len(df_answered)!=0:
        return len(df_answered_url)/len(df_answered)
    else:
        return 0

    
def get_df_url(conn):
    query = f'''
    SELECT tweets.tweet_id AS uniq, urls.tweet_id
    FROM tweets, urls
    WHERE tweets.tweet_id = urls.tweet_id
    '''
    df= pd.read_sql_query(query, conn).drop_duplicates()
    return df
    

In [8]:
def avg_time_response(air_id,avia_companies, df_all_replies, conn):
    df_mentioned = get_df_mentioned(air_id, conn)
    df_answered = get_df_answered(air_id, df_mentioned, df_all_replies)
    df_time_taken=df_answered.apply(lambda x: function_two(x.reply_id, x.reply_to_status_id, conn), axis=1).copy()
    average_time = df_time_taken.mean()
    seconds_time = average_time
    return seconds_time

def median_time_response(air_id,avia_companies, df_all_replies, conn):
    df_mentioned = get_df_mentioned(air_id, conn)
    df_answered = get_df_answered(air_id, df_mentioned, df_all_replies)
    df_time_taken=df_answered.apply(lambda x: function_two(x.reply_id, x.reply_to_status_id, conn), axis=1).copy()
    average_time = df_time_taken.median()
    seconds_time = average_time
    return seconds_time

def function_two(first_id, second_id, conn):
    query_one = f'''
    SELECT tweet_id, timestamp_ms
    FROM tweets
    WHERE tweet_id = {first_id}
    '''
    query_two = f'''
    SELECT tweet_id, timestamp_ms
    FROM tweets
    WHERE tweet_id = {second_id}
    '''
    df_one = pd.read_sql_query(query_one, conn)
    df_two = pd.read_sql_query(query_two, conn)
    timestamp_one = df_one['timestamp_ms'].values[0]
    timestamp_two = df_two['timestamp_ms'].values[0]
    return (timestamp_one - timestamp_two)//1000//60

def get_df_time_taken(air_id,avia_companies, df_all_replies, conn):
    df_mentioned = get_df_mentioned(air_id, conn)
    df_answered = get_df_answered(air_id, df_mentioned, df_all_replies)
    df_answered['time_taken']=df_answered.apply(lambda x: function_two(x.reply_id, x.reply_to_status_id, conn), axis=1).copy()
    return_df = df_answered[['tweet_id', 'time_taken', 'followers_count']].copy().reset_index().drop('index', axis='columns')
    return return_df

In [9]:
conn = create_connection(db_name)
avia_reply_percentages = {}
for key in reverse_avia_companies.keys():
    avia_reply_percentages[reverse_avia_companies[key]] = get_reply_perc(key, reverse_avia_companies, df_all_replies, conn)
avia_reply_percentages = dict(sorted(avia_reply_percentages.items(), key=lambda x:x[1], reverse=True))


2.6.0


In [10]:
conn = create_connection(db_name)
avia_url_percentages = {}
for key in reverse_avia_companies.keys():
    avia_url_percentages[reverse_avia_companies[key]] = get_url_perc(key, reverse_avia_companies, df_all_replies, conn)
avia_url_percentages = dict(sorted(avia_url_percentages.items(), key=lambda x:x[1], reverse=True))


2.6.0


In [11]:
conn = create_connection(db_name)
avia_time_answer_dfs = {}
avia_time_answer_median = {}
avia_time_answer_mean = {}
for key in reverse_avia_companies.keys():
    avia_time_answer_df = get_df_time_taken(key, reverse_avia_companies, df_all_replies, conn)
    avia_time_answer_median[reverse_avia_companies[key]] = avia_time_answer_df['time_taken'].median()
    avia_time_answer_mean[reverse_avia_companies[key]] = avia_time_answer_df['time_taken'].mean()
    avia_time_answer_dfs[reverse_avia_companies[key]] = avia_time_answer_df
avia_time_answer_mean = dict(sorted(avia_time_answer_mean.items(), key=lambda x:x[1], reverse=True))
avia_time_answer_median = dict(sorted(avia_time_answer_median.items(), key=lambda x:x[1], reverse=True))

2.6.0


In [21]:
avia_time_answer_dfs

,tweet_id,time_taken,followers_count
0,1131187978771025921,21,17
1,1131188089328734209,22,17
2,1131188287803142145,10,137
3,1131188303821180928,23,17
4,1131197267493445633,21,31
...,...,...,...
18420,1244683439426678796,11849,7
18421,1244683518287978496,11849,7
18422,1244688715806539776,28,51
18423,1244689736125624321,14,177


In [17]:
avia_time_answer_mean

{'British_Airways': 747.2228091198638,
 'RyanAir': 567.647079398391,
 'AirFrance': 452.85205747877205,
 'Qantas': 365.3804975566415,
 'easyJet': 293.2458643988816,
 'KLM': 246.85845318860245,
 'SingaporeAir': 228.24389508592103,
 'VirginAtlantic': 226.89251013694437,
 'AmericanAir': 54.76287268098893,
 'EtihadAirways': 35.349862258953166,
 'Lufthansa': 31.277857422129546}

In [82]:
avia_time_answer_median

{'British_Airways': 168.0,
 'Qantas': 113.0,
 'SingaporeAir': 71.0,
 'RyanAir': 51.0,
 'AirFrance': 49.0,
 'easyJet': 40.0,
 'AmericanAir': 29.0,
 'EtihadAirways': 17.5,
 'KLM': 14.0,
 'VirginAtlantic': 14.0,
 'Lufthansa': 7.0}

In [18]:
avia_url_percentages

{'easyJet': 0.508096458527493,
 'Lufthansa': 0.4496285677049394,
 'SingaporeAir': 0.3234850768766958,
 'Qantas': 0.32030208796090626,
 'RyanAir': 0.3192724728926198,
 'British_Airways': 0.3095613509507742,
 'EtihadAirways': 0.2493112947658402,
 'VirginAtlantic': 0.22607298599954098,
 'AirFrance': 0.22028086218158066,
 'KLM': 0.2194843962008141,
 'AmericanAir': 0.08304084545068847}

In [19]:
avia_reply_percentages

{'British_Airways': 0.463336766972772,
 'SingaporeAir': 0.4415895626705718,
 'KLM': 0.43857561114946086,
 'VirginAtlantic': 0.3920163152685721,
 'easyJet': 0.3707911276214337,
 'AmericanAir': 0.3565874769628563,
 'AirFrance': 0.291674604686607,
 'Qantas': 0.2897039897039897,
 'Lufthansa': 0.2644129708122265,
 'RyanAir': 0.1780709294069285,
 'EtihadAirways': 0.05779794602340578}

In [89]:
query = f'''
    SELECT tweets.tweet_id AS uniq, user_mentions.tweet_id, user_mentions.text, tweets.text, tweets.tweet_type, tweets.lang 
    FROM tweets, user_mentions
    WHERE tweets.tweet_id = user_mentions.tweet_id AND user_mentions.text = '38676903' AND tweets.lang != 'und'
    '''
df= pd.read_sql_query(query, conn).drop_duplicates()

In [90]:
df

,uniq,tweet_id,text,text,tweet_type,lang
0,1131173038194606081,1131173038194606081,38676903,@easyJet It is impossible to get in contact wi...,reply,en
1,1131173142414712832,1131173142414712832,38676903,And had to pay extra £50 because bag wouldnt f...,reply,en
2,1131173644271587328,1131173644271587328,38676903,Yes please @Ryanair @easyJet,quote,en
3,1131173747191369729,1131173747191369729,38676903,@easyJet El mio vuelu sofrió un retrasu d'unos...,reply,es
4,1131173839512264706,1131173839512264706,38676903,And had to pay extra £50 because bag wouldnt f...,retweet,en
...,...,...,...,...,...,...
386753,1244696059168919555,1244696059168919555,38676903,@ASuglani @easyJet I believe I called the UK 0...,retweet,en
386754,1244696257781805056,1244696257781805056,38676903,@easyJet I replied to your message at 10.17am ...,reply,en
386755,1244696371900436481,1244696371900436481,38676903,@lvpearcy32 @easyJet They haven't replied to m...,reply,en
386756,1244696491580628993,1244696491580628993,38676903,@adnansaleemiX @qatarairways @emirates @easyJe...,reply,en
